## 0 | Import packages

In [265]:
import os
import h5py
import numpy as np
import pandas as pd
import tkinter
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from utilities import importFile, openFile, openHDF5file, getLooseRseal
from tkinter.filedialog import askopenfilename, askopenfilenames
from collections import defaultdict
from nptdms import TdmsFile
from scipy import stats
from scipy.signal import find_peaks
print("done!")

done!


## 1 | Use find_peaks() to detect spikes in all sweeps

Now that we can detect spikes and are sure we are not counting noise, we can continue to do the same for all the sweeps of the cell we are analysing. We are going to need a few things. We are going to need to (1) be able to plot the histogram of prominences for the detected peaks and (2) be able to ask the user for input so it can set the adequate prominence bracket to detect spikes and not noise. And we need to do this on a sweep by sweep basis.

### 1.1 | Ask for input and plot within a loop

We can ask for input with the input() function, and convert the value to integer.

In [ ]:
prominence_min = int(input("Enter the min value for the desired prominence"))
prominence_max = int(input("Enter the max value for the desired prominence"))

print(prominence_min)
print(prominence_max)

We next need a way to first show the histogram, and then ask for the user input.

In [ ]:
%matplotlib tk
#%matplotlib inline

for i in range(10):
    plt.scatter(1, i)
    plt.show(block = True)
    plt.close()
    
    prominence_min = int(input("Enter the min value for the desired prominence"))
    prominence_max = int(input("Enter the max value for the desired prominence"))
    print(f'For sweep number {i}, you chose a prominence between {prominence_min} and {prominence_max}')

Would it work if we just concatenate all sweeps?

In [ ]:
c = np.array(channels_data_frame.loc['Channel B', :])
sweep_IB_concatenated = np.concatenate(c)

%matplotlib tk

plt.figure(1)
plt.plot(sweep_IB_concatenated, 'k')
#plt.xlim(left=test_pulse_IB_end)
plt.title('Figure 4.1: Concatenated Sweeps', fontsize = 14)
plt.ylabel('current [pA]')
plt.show()

In [ ]:
Rseal_data_frame = getLooseRseal_bumped(file_name, channels_data_frame)
Rseal_data_frame

In [ ]:
plt.hist(np.array(Rseal_data_frame.loc['test_pulse_membrane', :]), bins = 50, density = False, histtype = 'bar', log = False)
plt.title('Figure 4.2: Membrane response to test_pulse', fontsize = 14)
plt.xlabel('current [pA]', fontsize = 12)
plt.show()

In [ ]:
peaks_concatenated, properties_concatenaded = find_peaks(-sweep_IB_concatenated, height = (None, None), threshold = (None, None), distance = None, prominence = (None, None), width = (None, None))
# Reverse the sign of the data to detect the lower peak of the spike (some are unipolar and only go down) and to avoid the noise.
print(len(peaks_concatenated))
peaks_concatenated

In [ ]:
plt.hist(properties_concatenaded['widths'], bins = 50, density = False, histtype = 'bar', log = True)
plt.title('Figure 4.3: Prominence of detected peaks', fontsize = 14)
plt.xlabel('peak prominence [pA]', fontsize = 12)
plt.show()

In [ ]:
np.where(properties_concatenaded['widths'] > 2000)

In [ ]:
%matplotlib tk

# Set colormap
cmap = plt.get_cmap('Pastel2')
noise_color = cmap(0)
spikes_color = cmap(1)

# Define region corresponding to spike size (pA)
spike_size_left_edge = 120
spike_size_right_edge = 240

n, bins, patches = plt.hist(properties_concatenaded['prominences'], bins = 200, density = False, histtype = 'bar', log = True)
# Use log = True to see the smaller peak in the histogram that corresponds to the spikes

for i in range(len(patches)):
    if (spike_size_left_edge < bins[i] < spike_size_right_edge):
        patches[i].set_facecolor(spikes_color)
    else:
        patches[i].set_facecolor(noise_color)

plt.title('Figure 4.4: Prominence of detected peaks', fontsize = 14)
plt.xlabel('peak prominence [pA]', fontsize = 12)
plt.show()

In [ ]:
peaks, properties = find_peaks(-sweep_IB_concatenated, height = (0.5), threshold = (None, None), distance = None, prominence = (Nonee), width = (None(None0), wlen = (10/dt))
# Reverse the sign of the data to detect the lower peak of the spike (some are unipolar and only go down) and to avoid the noise.
print(len(peaks))
peaks

In [ ]:
sum(properties_concatenaded['prominences'] > 232)

In [ ]:
plt.plot(peaks, sweep_IB_concatenated[peaks >232], "or")

In [ ]:
%matplotlib tk

plt.plot(peaks, sweep_IB_concatenated[peaks], "xr"; plt.plot(sweep_IB_concatenated); plt.legend(['peaks'])
plt.title('Figure 4.5: Detected peaks', fontsize = 14)
plt.xlabel('samples', fontsize = 12)
plt.ylabel('current [pA]', fontsize = 12)
plt.show()

In [ ]:
len(peaks) / (len(sweep_IB_concatenated)*dt*1000)

### 1.2. | Find spikes on a sweep by sweep basis

Let's first remind ourselves of the data frame shape.


In [ ]:
# Load data
channels_data_frame, time, dt, folder_name, file_name = importFile(curated_channel = 'Sweeps_Analysis')
print("file imported")

In [ ]:
channels_data_frame

In [ ]:
# pandas.DataFrame.columns - The column labels of the DataFrame.
print(channels_data_frame.columns)

# pandas.DataFrame.index - The index (row labels) of the DataFrame.
print(channels_data_frame.index)

# pandas.DataFrame.iloc - integer-location based indexing for selection by position.
channels_data_frame.loc[['Channel A', 'Channel B'],:]

In [ ]:
for sweep in channels_data_frame.columns:
    sweep_IB = np.array(channels_data_frame.at['Channel B', sweep])

    peaks, properties = find_peaks(-sweep_IB,
                                   height = None,
                                   threshold = None,
                                   distance = 5,
                                   prominence = 50,
                                   width = 2,
                                   wlen = None,
                                   rel_height = None,
                                   plateau_size = None)
                                

In [ ]:
    cell_name = [file_name.split('.')[0]] # Get the file name without the extension
    print(cell_name)
    seal_resistance = []
    trial_keys = []

    for sweep in channels_data_frame.columns:
        
        sweep_IA = np.array(channels_data_frame.at['Channel A', sweep])
        sweep_IB = np.array(channels_data_frame.at['Channel B', sweep])
        sweep_OA = np.array(channels_data_frame.at['Output A', sweep])

        # Get the indices of the test_pulse using the Output Channel
        test_pulse = np.where(sweep_OA < 0)
        test_pulse_OA_indices = test_pulse[0]

        # # Find the edges of the test_pulse
        # test_pulse_start = np.where(np.diff(sweep_OA) < (0))
        # test_pulse_end = np.where(np.diff(sweep_OA) > (0))
        # # Extract them from the tuple
        # test_pulse_start_i = test_pulse_start[0][0]
        # test_pulse_end_i = test_pulse_end[0][0]

        # Use the indices of the test_pulse command to define baseline period and test period
        sweep_OA_baseline = np.mean(sweep_OA[:(test_pulse_OA_indices[0]-1)]) # -1 to stop baseline before command starts
        sweep_OA_pulse = np.mean(sweep_OA[test_pulse_OA_indices])
        test_pulse_command = sweep_OA_baseline - sweep_OA_pulse # mV

        # # Use the indices of the test_pulse command to define baseline period and test period
        # sweep_OA_baseline = np.mean(sweep_OA[int(test_pulse_start_i - (20/0.04)):test_pulse_start_i])
        # sweep_OA_pulse = np.mean(sweep_OA[int(test_pulse_end_i - (49/0.04)):test_pulse_end_i])
        # test_pulse_command = sweep_OA_baseline - sweep_OA_pulse #mV


        # Use the test_pulse indices to get the baseline and cell response to calculate the seal resistance
        # To be exact and account for the delays between digital command and output from the amplifier, you could add +1 to the first index to calculate the baseline.
        sweep_IB_baseline = np.mean(sweep_IB[:(test_pulse_OA_indices[0])])
        # Similary, to avoid using the values recorded while the test pulse command begins, you can skip a milisecond (+4 indices) to the beginning, to ensure you start averaging once the signal has reached the cell. To be extra exact, you could add +2 to the last index so you use all the samples. However, this shouldn't make a difference, so we just skip the milisecond to avoid the transition period.
        sweep_IB_pulse = np.mean(sweep_IB[(test_pulse_OA_indices[0]+4):(test_pulse_OA_indices[-1])])
        test_pulse_membrane = sweep_IB_baseline - sweep_IB_pulse # pA

        # # Do the same as above but for the Current channel
        # sweep_IB_baseline = np.mean(sweep_IB[int(test_pulse_start_i - (20/0.04)):test_pulse_start_i])
        # sweep_IB_pulse = np.mean(sweep_IB[int(test_pulse_end_i - (49/0.04)):test_pulse_end_i])
        # test_pulse_membrane = sweep_IB_baseline - sweep_IB_pulse #pA

        # Get seal resistance = mV/pA
        Rseal = (test_pulse_command / test_pulse_membrane) * 1000 # to get MOhm
        
        # append results
        seal_resistance.append(Rseal)
        trial_keys.append(sweep)

    # Create data frame of data:
    extracted_Rseal_data_frame = pd.DataFrame([seal_resistance], index = cell_name, columns = trial_keys)


# Types of firing rate to assess

 * Instantaneous firing rate: inverse of the interspike interval

 * Firing rate over full recording

 * Firing rate over time windows (1s?)